## Example showing generalized processing of commuting terms

In [1]:
from qiskit.quantum_info import Pauli, SparsePauliOp
from itertools import combinations
import numpy as np

def pauli_commutes(pauli1, pauli2):
    """
    Check if two Pauli operators commute using their string representations.
    
    Args:
    pauli1 (SparsePauliOp): First Pauli operator.
    pauli2 (SparsePauliOp): Second Pauli operator.
    
    Returns:
    bool: True if pauli1 and pauli2 commute, False otherwise.
    """
    pauli_str1 = pauli1.paulis[0].to_label()
    pauli_str2 = pauli2.paulis[0].to_label()
    
    for i in range(len(pauli_str1)):
        if pauli_str1[i] != 'I' and pauli_str2[i] != 'I' and pauli_str1[i] != pauli_str2[i]:
            return False
    return True

def build_commutativity_graph(pauli_list):
    """
    Build a commutativity graph where nodes represent Pauli operators,
    and edges represent commutation relations between them.
    
    Args:
    pauli_list (list of SparsePauliOp): List of Pauli operators.
    
    Returns:
    np.ndarray: Adjacency matrix of the commutativity graph.
    """
    num_terms = len(pauli_list)
    adjacency_matrix = np.zeros((num_terms, num_terms), dtype=bool)
    
    for i, j in combinations(range(num_terms), 2):
        if pauli_commutes(pauli_list[i], pauli_list[j]):
            adjacency_matrix[i, j] = True
            adjacency_matrix[j, i] = True
    
    return adjacency_matrix

def group_commuting_terms(pauli_list):
    """
    Group commuting Pauli terms.
    
    Args:
    pauli_list (list): List of tuples where each tuple contains a Pauli string and a coefficient.
    
    Returns:
    list: List of groups where each group is a list of commuting Pauli terms.
    """
    # Convert the list of Pauli strings to SparsePauliOp objects
    paulis = [SparsePauliOp.from_list([(p, 1)]) for p, coeff in pauli_list]
    
    # Build the commutativity graph
    adjacency_matrix = build_commutativity_graph(paulis)
    
    # Group commuting terms
    groups = []
    ungrouped_indices = set(range(len(paulis)))
    
    while ungrouped_indices:
        current_group = []
        i = ungrouped_indices.pop()
        current_group.append(pauli_list[i])
        
        for j in ungrouped_indices.copy():
            if adjacency_matrix[i, j]:
                current_group.append(pauli_list[j])
                ungrouped_indices.remove(j)
        
        groups.append(current_group)
    
    return groups

# Example usage
hamiltonian = [
    ('XXII', 0.5),
    ('IYYI', 0.3),
    ('IIZZ', 0.4),
    ('XYII', 0.2),
    ('IIYX', 0.6),
    ('IZXI', 0.1),
    ('XIII', 0.7)
]

groups = group_commuting_terms(hamiltonian)
for i, group in enumerate(groups):
    print(f"Group {i+1}:")
    for pauli, coeff in group:
        print(f"  {pauli}: {coeff}")


Group 1:
  XXII: 0.5
  IIZZ: 0.4
  IIYX: 0.6
  XIII: 0.7
Group 2:
  IYYI: 0.3
  XYII: 0.2
Group 3:
  IZXI: 0.1


## Compute the energy

In [2]:
from qiskit import QuantumCircuit,transpile, assemble
from qiskit_aer import Aer
import numpy as np

# Initialize the backend and the simulator
backend = Aer.get_backend('qasm_simulator')


groups = [
    [('XXII', 0.5), ('XYII', 0.2), ('XIII', 0.7)],
    [('IYYI', 0.3), ('IIYX', 0.6)],
    [('IIZZ', 0.4), ('IZXI', 0.1)]
]

# Function to create circuits for each group
def create_circuits(groups):
    circuits = []
    for group in groups:
        qc = QuantumCircuit(4)
        for term, coeff in group:
            for i, pauli in enumerate(term):
                if pauli == 'X':
                    qc.h(i)
                elif pauli == 'Y':
                    qc.sdg(i)
                    qc.h(i)
        qc.measure_all()
        circuits.append((qc, group))
    return circuits

# Create the circuits
circuits = create_circuits(groups)



# Function to calculate expectation values from results
def calculate_expectation(results, circuits):
    total_energy = 0
    for (qc, group), result in zip(circuits, results.get_counts()):
        #counts = result.get_counts()
        counts = result
        for term, coeff in group:
            # Calculate the expectation value for each term
            exp_val = 0
            for bitstring, count in counts.items():
                parity = (-1)**sum([int(bitstring[i]) for i, pauli in enumerate(term) if pauli != 'I'])
                exp_val += parity * count
            exp_val /= sum(counts.values())
            total_energy += coeff * exp_val
    return total_energy

# Calculate the total energy
for i in range(10):

    # Compile and execute the circuits
    transpiled_circuits = transpile([circuit for circuit, group in circuits], backend)
    #qobj = assemble(transpiled_circuits)
    results = backend.run(transpiled_circuits).result()

    total_energy = calculate_expectation(results, circuits)
    print(f"Total Energy: {total_energy}")


Total Energy: 1.777734375
Total Energy: 1.805859375
Total Energy: 1.8242187499999998
Total Energy: 1.7757812499999999
Total Energy: 1.8080078125
Total Energy: 1.7931640625000003
Total Energy: 1.8205078124999998
Total Energy: 1.7925781249999997
Total Energy: 1.778125
Total Energy: 1.834375
